In [0]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

from google.colab import drive
drive.mount('/content/drive')
text = open('/content/drive/My Drive/data_smoking_status_challenge_2006.txt', 'rb').read().decode(encoding='utf-8')
text_seed = open('/content/drive/My Drive/data_medication_challenge_2009.txt', 'rb').read().decode(encoding='utf-8')
print('train data corpus length:', len(text))
print('seed data corpus length:', len(text_seed))

chars_train = sorted(list(set(text)))
print('total chars_train:', len(chars_train))
chars_seed = sorted(list(set(text_seed)))
print('total chars_seed:', len(chars_seed))

chars=list(set().union(chars_train,chars_seed))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 200
step = 15
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text_seed) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text_seed[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(1000):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=10,
          callbacks=[print_callback])

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
train data corpus length: 3199629
seed data corpus length: 4857819
total chars_train: 89
total chars_seed: 94
total chars: 95
nb sequences: 213296
Vectorization...
Build model...
Epoch 1/10
213296/213296 [==============================] - 797s 4ms/step - loss: 2.0502

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: ".8 , pulse 101 , blood pressure 130/90.
Respiratory rate 20 , saturation 95% on room air. Weight 175
pounds.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:62: RuntimeWarning: divide by zero encountered in log


ssulted
DIS
Admission Date :
04/08/200d
CUR DISCHARGE DIAGNOSIS :
No Resultation Indication
DIS
Admissien Date :
07/08/200d
CUR HISTORY :
No Restrictions
DISCHARGE DIAGNOSIS :
After for continued to the patient was started on the removal status post continued to the patient was admitting with a continued to the patient was bilaterally .
The patient was admitting with status post repair on the patient was admitting on the patient was placement .
PAST MEDICAL HISTORY :
On postoperative in her physical hospital and with a progress .
The patient was discharge .
He was admitted to the patient was placed on 03/08/00 .
Previon of her cardiac on the right septime .
ADDITIONAL COMMENTS :
The patient was continued to the patient was admitting to the patient was admitting with a head , which was discharge .
She had a region .
At that the patient discontinued to the patient was decreased on the patient was admitting on the patient was 
----- diversity: 0.5
----- Generating with seed: "se see above

KeyboardInterrupt: ignored